## Directly reading as a graph

In [1]:
from neo4j import GraphDatabase
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
import getpass
import os
import os


graph = Neo4jGraph(
    url="bolt://54.205.44.248:7687",
    username="neo4j",
    password="hat-certificates-closures"
)

name = 'OPENAI_API_KEY'
secret = getpass.getpass("Enter the OpenAI API Key")
os.environ[name] = secret
print(f"Have successfully set up {name} as an environment variable.")

Have successfully set up OPENAI_API_KEY as an environment variable.


In [2]:
# Initialize connection to Neo4j database
uri = "bolt://54.205.44.248:7687"  # Replace with your Neo4j URI
auth = ("neo4j", "hat-certificates-closures")  # Replace with your Neo4j username and password
driver = GraphDatabase.driver(uri, auth=auth)

In [3]:
# Setting up ChatGPT integration with Graph Database
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True,
)

In [4]:
# Success
chain.run("""
How many nodes do we have?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n) RETURN count(n)
Full Context:
[{'count(n)': 7}]

> Finished chain.


'We have 7 nodes.'

In [60]:
# Success
chain.run("""
How many other nodes does each node connect to?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Loc)-[r:ROAD]->(m:Loc)
RETURN n.name, COUNT(DISTINCT m) AS num_connections
Full Context:
[{'n.name': 'A', 'num_connections': 3}, {'n.name': 'B', 'num_connections': 1}, {'n.name': 'C', 'num_connections': 2}, {'n.name': 'D', 'num_connections': 3}, {'n.name': 'E', 'num_connections': 1}]

> Finished chain.


'Node A connects to 3 other nodes. Node B connects to 1 other node. Node C connects to 2 other nodes. Node D connects to 3 other nodes. Node E connects to 1 other node.'

In [61]:
# Success
# You must give model context about the relationship property
chain.run("""
What is the path with the lowest cost from A to F? The relationship property between each node is the cost of the path between the nodes.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH path = (start:Loc {name: 'A'})-[:ROAD*]->(end:Loc {name: 'F'})
WITH path, reduce(totalCost = 0, r in relationships(path) | totalCost + r.cost) AS pathCost
RETURN path
ORDER BY pathCost ASC
LIMIT 1
Full Context:
[{'path': [{'name': 'A'}, 'ROAD', {'name': 'C'}, 'ROAD', {'name': 'D'}, 'ROAD', {'name': 'E'}, 'ROAD', {'name': 'F'}]}]

> Finished chain.


'The path with the lowest cost from A to F is A -> C -> D -> E -> F.'

In [71]:
# Success
chain.run("""
Are there any cycles within the graph? Identify all cycles.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH p=(n)-[:ROAD*]->(n)
RETURN p
Full Context:
[{'p': [{'name': 'A'}, 'ROAD', {'name': 'D'}, 'ROAD', {'name': 'A'}]}, {'p': [{'name': 'A'}, 'ROAD', {'name': 'C'}, 'ROAD', {'name': 'D'}, 'ROAD', {'name': 'A'}]}, {'p': [{'name': 'A'}, 'ROAD', {'name': 'B'}, 'ROAD', {'name': 'D'}, 'ROAD', {'name': 'A'}]}, {'p': [{'name': 'B'}, 'ROAD', {'name': 'D'}, 'ROAD', {'name': 'A'}, 'ROAD', {'name': 'B'}]}, {'p': [{'name': 'C'}, 'ROAD', {'name': 'D'}, 'ROAD', {'name': 'A'}, 'ROAD', {'name': 'C'}]}, {'p': [{'name': 'D'}, 'ROAD', {'name': 'A'}, 'ROAD', {'name': 'D'}]}, {'p': [{'name': 'D'}, 'ROAD', {'name': 'A'}, 'ROAD', {'name': 'C'}, 'ROAD', {'name': 'D'}]}, {'p': [{'name': 'D'}, 'ROAD', {'name': 'A'}, 'ROAD', {'name': 'B'}, 'ROAD', {'name': 'D'}]}]

> Finished chain.


'Yes, there are cycles within the graph. The cycles are as follows:\n- A -> D -> A\n- A -> C -> D -> A\n- A -> B -> D -> A\n- B -> D -> A -> B\n- C -> D -> A -> C\n- D -> A -> D\n- D -> A -> C -> D\n- D -> A -> B -> D'

In [47]:
# Success but with errors
# This method is only valid if the answer we're looking for can be answered with a cypher statement
chain.run("""
What type of graph is this?
          """)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
This is a graph with nodes representing locations and relationships representing roads between locations.


ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'This': expected
  "ALTER"
  "CALL"
  "CREATE"
  "DEALLOCATE"
  "DELETE"
  "DENY"
  "DETACH"
  "DROP"
  "DRYRUN"
  "ENABLE"
  "FOREACH"
  "GRANT"
  "LOAD"
  "MATCH"
  "MERGE"
  "OPTIONAL"
  "REALLOCATE"
  "REMOVE"
  "RENAME"
  "RETURN"
  "REVOKE"
  "SET"
  "SHOW"
  "START"
  "STOP"
  "TERMINATE"
  "UNWIND"
  "USE"
  "USING"
  "WITH" (line 1, column 1 (offset: 0))
"This is a graph with nodes representing locations and relationships representing roads between locations."
 ^}

In [68]:
# Fail
# Bad query
chain.run("""
How many routes are there from node A to D? Do not go through the same node more than once.
          """)

# Good query after using ChatGPT 4

'''
MATCH path = (a:Loc {name: 'A'})-[:ROAD*]->(d:Loc {name: 'D'})
WHERE ALL(n IN nodes(path) WHERE 1 = size([m IN nodes(path) WHERE m = n]))
RETURN count(DISTINCT path) AS num_routes
'''



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Loc {name: 'A'})-[:ROAD*]->(d:Loc {name: 'D'})
WHERE ALL(n IN nodes(path) WHERE size(filter(m IN nodes(path) WHERE m = n)) = 1)
RETURN count(distinct path) AS num_routes


ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'WHERE': expected
  "!="
  "%"
  ")"
  "*"
  "+"
  ","
  "-"
  "/"
  "<"
  "<="
  "<>"
  "="
  ">"
  ">="
  "AND"
  "OR"
  "XOR"
  "^" (line 2, column 63 (offset: 118))
"WHERE ALL(n IN nodes(path) WHERE size(filter(m IN nodes(path) WHERE m = n)) = 1)"
                                                               ^}

In [76]:
# Success-ish
# Cypher query is correct and output is correct
# LLM unable to summarize
# Caveat: must use the word "path"
chain.run("""
Out of all the path from A to D, which one has the lowest cost? The cost of the route between 2 nodes is the relationship property between those 2 nodes.""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH path = (start:Loc {name: 'A'})-[:ROAD*]->(end:Loc {name: 'D'})
RETURN path
ORDER BY reduce(totalCost = 0, r in relationships(path) | totalCost + r.cost) ASC
LIMIT 1
Full Context:
[{'path': [{'name': 'A'}, 'ROAD', {'name': 'C'}, 'ROAD', {'name': 'D'}]}]

> Finished chain.


"I'm sorry, but I don't have enough information to determine the cost of each path from A to D."

In [34]:
def get_node_count(tx):
    result = tx.run("MATCH (n) RETURN COUNT(n) AS node_count")
    return result.single()["node_count"]

# Query the database to get the node count
with driver.session() as session:
    node_count = session.read_transaction(get_node_count)
    print(f'There are {node_count} nodes in the database.')

There are 6 nodes in the database.


/tmp/ipykernel_2030018/1610171175.py:7: DeprecationWarning: read_transaction has been renamed to execute_read
  node_count = session.read_transaction(get_node_count)


In [69]:
1.5/2.25

0.6666666666666666